In [1]:
from brainstem_api_tools.brainstem_api_client import BrainstemClient

import json

# 1. Client Setup and Authentication
The Brainstem API client provides easy access to the Brainstem data platform. To get started, initialize the client, which will prompt you for login credentials the first time. After successful authentication, the client will generate a token that can be saved for future use. This allows you to avoid re-entering your credentials each time.

In [ ]:
# When initializing client without a token
client = BrainstemClient()

# 2. Loging in with token
For convenience, you can save your authentication token to a configuration file. This allows you to quickly initialize the client in future sessions without entering login details. Simply load the token from your saved file and pass it to the client constructor.

In [ ]:
token = None # Input your token here
if token:
    client = BrainstemClient(token=token)
    print("Client initialized with saved token")
else:
    # This will prompt for username/password if run
    print("No saved token found. Will need to login.")
    client = BrainstemClient()

# 3. Loading Data Models (Sessions Example - Public Data)

The Brainstem platform hosts both public and private data. Public data can be accessed without authentication. Use the portal="public" parameter to specifically query public repositories. This is useful for exploring datasets like the Allen Institute's Visual Coding – Neuropixels project.

In [4]:
# Load public projects (no authentication needed)
public_projects = client.load_model("project", portal="public").json()

# Print the number of available public projects
print(f"Found {len(public_projects.get('projects', []))} public projects")

# Display the first few projects
print("\nSample public projects:")
for i, project in enumerate(public_projects.get('projects', [])[:3]):
    print(f"{i+1}. {project.get('name', 'Unnamed')}")
    print(f"   Description: {project.get('description', 'No description')[:100]}...")
    print("")



Found 18 public projects

Sample public projects:
1. interaction of acetylcholine and oxytocin neuromodulation in the hippocampus
   Description: <p>A postulated role of subcortical neuromodulators is to control brain states. Mechanisms by which ...

2. Lore's first project
   Description: <p>Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labo...

3. Metal microdrive and head cap system for silicon probe recovery in freely moving rodent
   Description: <p>High-yield electrophysiological extracellular recording in freely moving rodents provides a uniqu...



In [ ]:
# To load private projects (requires authentication):
"""
# Load your private projects (requires valid token)
private_projects = client.load_model("project").json() # default is private

# Print the number of your private projects
print(f"Found {len(private_projects.get('projects', []))} private projects")

# Display your private projects
print("\nYour private projects:")
for i, project in enumerate(private_projects.get('projects', [])[:3]):
    print(f"{i+1}. {project.get('name', 'Unnamed')}")
    print(f"   Created by: {project.get('principal_investigator', 'Unknown')}")
"""

# 4. Filtering

The API supports powerful filtering capabilities to help you find specific records. You can filter by exact matches, partial text matches (case-sensitive or insensitive), and various other criteria. Multiple filters can be combined to narrow your search. Common filter modifiers include .icontains for case-insensitive text search and .iexact for exact matching.

### Filter Modifiers

You can use the following filter modifiers to refine your search:

- `attribute.contains`: Case-sensitive partial match
- `attribute.icontains`: Case-insensitive partial match
- `attribute.iexact`: Case-insensitive exact match
- `attribute.startswith`: Case-sensitive prefix match
- `attribute.endswith`: Case-sensitive suffix match
- `attribute.istartswith`: Case-insensitive prefix match
- `attribute.iendswith`: Case-insensitive suffix match

In [5]:
# Basic filtering example
filtered_projects = client.load_model(
    "project",
    portal="public",
    filters={'name.icontains': 'Allen'}  # Case-insensitive contains
).json()

print(f"Found {len(filtered_projects.get('projects', []))} projects matching 'Allen'")

# Multiple filters with AND logic
multi_filter = client.load_model(
    "project",
    portal="public",
    filters={
        'name.icontains': 'institute',
        'name.iendswith': 'neuropixels'  # Fixed typo in operator name
    }
).json()

if isinstance(multi_filter, dict):
    count = len(multi_filter.get('projects', []))
else:
    count = 0

print(f"Found {count} projects matching all criteria")

print("\nResponse type:", type(multi_filter))
if isinstance(multi_filter, dict) and 'projects' in multi_filter:
    print("Response contains 'projects' key with data")
    print(f"The id of project is {multi_filter['projects'][0]['id']} and the name is {multi_filter['projects'][0]['name']}")

Found 1 projects matching 'Allen'
Found 1 projects matching all criteria

Response type: <class 'dict'>
Response contains 'projects' key with data
The id of project is df3c7a20-c03e-48fc-a65d-613cdf1deff9 and the name is Allen Institute: Visual Coding – Neuropixels


# 5. Basic Data Retrieval 

In [6]:
allen_projects = client.load_model(
    "project", 
    portal="public", 
    filters={'name.iexact': 'Allen Institute: Visual Coding – Neuropixels'}
).json()
print(allen_projects['projects'][0])

{'id': 'df3c7a20-c03e-48fc-a65d-613cdf1deff9', 'name': 'Allen Institute: Visual Coding – Neuropixels', 'description': '<p>The Visual Coding &ndash; Neuropixels project uses high-density extracellular electrophysiology (<strong>Ecephys</strong>) probes to record spikes from a wide variety of regions in the mouse brain. Our experiments are designed to study the activity of the visual cortex and thalamus in the context of passive visual stimulation, but these data can be used to address a wide variety of topics. Allen Brain Observatory Neuropixels dataset; &copy;2019 Allen Institute for Brain Science <a href="https://allensdk.readthedocs.io/en/latest/visual_coding_neuropixels.html">https://allensdk.readthedocs.io/en/latest/visual_coding_neuropixels.html</a></p>', 'sessions': ['6a9c60e9-9a34-4da7-94bc-5bc9da15256f', '017b9289-26f6-4c28-be84-aee554125cd0', '8bab81b4-c37a-445a-b58c-08a19da1479a', '4454e33a-fe89-4745-9e57-9ff5fe561211', 'a452cd24-4b1a-472d-8790-286653e9932f', '857b4729-86ff-4

In [7]:
# Find the Allen Institute: Visual Coding – Neuropixels project
allen_projects = client.load_model(
    "project", 
    portal="public", 
    filters={'name.iexact': 'Allen Institute: Visual Coding – Neuropixels'}
).json()

project_id = allen_projects['projects'][0]['id']

# Get basic project information without including sessions
project_details = client.load_model(
    "project",
    portal="public",
    id=project_id
).json()

# Access project information
project = project_details['project']
print(f"Project name: {project['name']}")
print(f"Description: {project['description'][:200]}...")

# Get subjects related to this project
subjects = client.load_model(
    "subject", 
    portal="public",
    filters={'projects': project_id}
).json()

print(f"\nThis project has {len(subjects['subjects'])} subjects")

# Access the sessions included with the project - sessions should be available now
if 'sessions' in project:
    session_ids = project['sessions']
    print(f"This project has {len(session_ids)} sessions")
    
    # Get details for the first session as an example
    if session_ids:
        first_session_id = session_ids[0]
        session_details = client.load_model(
            "session",
            portal="public",
            id=first_session_id
        ).json()
        
        # Print session details
        if 'session' in session_details:
            session = session_details['session']
            print(f"\nExample session details:")
            print(f"ID: {first_session_id}")
            print(f"Name: {session['name']}")
else:
    print("No sessions found in project data")

Project name: Allen Institute: Visual Coding – Neuropixels
Description: <p>The Visual Coding &ndash; Neuropixels project uses high-density extracellular electrophysiology (<strong>Ecephys</strong>) probes to record spikes from a wide variety of regions in the mouse brain....

This project has 58 subjects
This project has 58 sessions

Example session details:
ID: 6a9c60e9-9a34-4da7-94bc-5bc9da15256f
Name: session_798911424


# 6. Including Related Models

Brainstem's data models have relationships between them (projects contain sessions, sessions involve subjects, etc.). The include parameter lets you fetch related models in a single request, reducing the number of API calls needed. For example, you can retrieve a project along with all its sessions and subjects in one operation

In [8]:
# Find the Allen Institute: Visual Coding – Neuropixels project
allen_projects = client.load_model(
    "project", 
    portal="public", 
    filters={'name.iexact': 'Allen Institute: Visual Coding – Neuropixels'}
).json()

# Get project ID
project_id = allen_projects['projects'][0]['id']

# Get detailed project information with sessions and subjects included
project_with_data = client.load_model(
    "project",
    portal="public",
    id=project_id,
    include=["sessions", "subjects"]  # Include both sessions and subjects
).json()

# Access project information
project = project_with_data['project']
print(f"Project name: {project['name']}")
print(f"Description: {project['description'][:100]}...")
print(f"Public: {project['is_public']}")

# Print session and subject counts. We have this data from a single API call
print(f"\nThis project has {len(project['sessions'])} sessions")
print(f"This project has {len(project['subjects'])} subjects")

# List first 3 subjects directly from the include
print("\nFirst 3 subjects in the project:")
for i, subject_id in enumerate(project['subjects'][:3]):
    print(f"{i+1}. Subject ID: {subject_id}")

# List first 3 sessions directly from the include
print("\nFirst 3 sessions in the project:")
for i, session_id in enumerate(project['sessions'][:3]):
    print(f"{i+1}. Session ID: {session_id}")

# Get details for a specific session with included dataacquisition
first_session_id = project['sessions'][0]
session_with_data = client.load_model(
    "session",
    portal="public",
    id=first_session_id,
    include=["dataacquisition"]  # Include data acquisition methods
).json()

# Access the session data
session = session_with_data['session']
print(f"\nSession details:")
print(f"Name: {session['name']}")
print(f"Description: {session['description']}")

# Now actually use the included dataacquisition data
if 'dataacquisition' in session:
    data_acq_ids = session['dataacquisition']
    print(f"\nThis session has {len(data_acq_ids)} data acquisition method(s) (included in response)")
    
    # Demonstrate using the included dataacquisition IDs
    if data_acq_ids:
        print("\nData acquisition IDs from session include:")
        for i, acq_id in enumerate(data_acq_ids):
            print(f"{i+1}. {acq_id}")

Project name: Allen Institute: Visual Coding – Neuropixels
Description: <p>The Visual Coding &ndash; Neuropixels project uses high-density extracellular electrophysiology (...
Public: True

This project has 58 sessions
This project has 58 subjects

First 3 subjects in the project:
1. Subject ID: 5c786545-ac38-4400-8aac-ef6c60632c91
2. Subject ID: c3e9d58e-501f-4654-b8a4-b3c3be9bbc68
3. Subject ID: 72c39529-e603-47d6-a5e6-8e71d08f21f1

First 3 sessions in the project:
1. Session ID: 6a9c60e9-9a34-4da7-94bc-5bc9da15256f
2. Session ID: 017b9289-26f6-4c28-be84-aee554125cd0
3. Session ID: 8bab81b4-c37a-445a-b58c-08a19da1479a

Session details:
Name: session_798911424
Description: <p>brain_observatory_1.1</p>

This session has 1 data acquisition method(s) (included in response)

Data acquisition IDs from session include:
1. 65d48097-cd83-4dfb-a8a1-b8f150ba85c7


# 7. Sorting

You can control the order of returned results using the sort parameter. Sort by any field in ascending order (e.g., alphabetically by name) or use a minus sign prefix for descending order.

In [9]:
# Sort projects alphabetically by name
alpha_projects = client.load_model(
    "project",
    portal="public",
    sort=['name']  # Ascending alphabetical order
).json()

print("\nAlphabetically sorted projects:")
for i, project in enumerate(alpha_projects.get('projects', [])[:3]):
    print(f"{i+1}. {project.get('name')}")

# Sort projects reverse-alphabetically by name
reverse_alpha = client.load_model(
    "project",
    portal="public",
    sort=['-name']  # Descending alphabetical order
).json()

print("\nReverse alphabetically sorted projects:")
for i, project in enumerate(reverse_alpha.get('projects', [])[:3]):
    print(f"{i+1}. {project.get('name')}")


Alphabetically sorted projects:
1. Allen Institute: Visual Coding – Neuropixels
2. CRCNS Frontal cortex: fcx-1
3. CRCNS Hippocampus: hc-1

Reverse alphabetically sorted projects:
1. Theta rhythm perturbation by focal cooling of the septal pacemaker in awake rats
2. Physiological Properties and Behavioral Correlates of Hippocampal Granule Cells and Mossy Cells
3. Nick Steinmetz Neuropixel data


# 8. Updating data
The API supports creating new records and updating existing ones. When creating a record, provide the required fields for that model type. For updates, fetch the existing record, modify its attributes, and save it back. Both operations require appropriate permissions.

In [ ]:
# Example of updating a session (use your own private data)
# Note: This is just syntax demonstration and won't execute on public data
token = None # Input your token here
if token:
    client = BrainstemClient(token=token)
    print("Client initialized with saved token")
else:
    # This will prompt for username/password if run
    print("No saved token found. Will need to login.")
    client = BrainstemClient()
"""
# First load a session you have permission to modify
filtered_session = client.load_model('session', filters={'name.iexact': 'your session'}).json()

# Your existing code works without using a 'session' key
filtered_session['description'] = 'This is a test description, showing how to update a session'

# Pass the whole filtered_session object to save_model
updated_session = client.save_model(
    'session', 
    id=filtered_session['sessions'][0]['id'], 
    data=filtered_session
).json()

print("Session updated successfully")
"""

# Creating a new subject - SYNTAX DEMONSTRATION ONLY
"""
# Create a new subject
new_subject_data = {
    'name': 'Test Subject 001',
    'species': 'mouse',
    'sex': 'M',
    'projects': ['your-project-id']  # Project this subject belongs to
}

# Save the new subject
new_subject = client.save_model(
    'subject',
    data=new_subject_data
).json()

print(f"New subject created with ID: {new_subject['subject']['id']}")
"""

print("To use them, replace placeholder IDs with your own private data IDs.")
print("Only run these commands on data you have permission to modify.")

# 9. Deleting data
You can remove records from the database using their IDs. Deletion is permanent, so use this operation with caution. The API will return a success status code (204) when deletion is successful. Like other write operations, deletion requires appropriate permissions.

In [ ]:
token = None # Input your token here.
if token:
    client = BrainstemClient(token=token)
    print("Client initialized with saved token")
else:
    # This will prompt for username/password if run
    print("No saved token found. Will need to login.")
    client = BrainstemClient()

# Example of deleting a session (demonstration only)
"""
# First create a test session that we can safely delete
new_session_data = {
    'name': 'Temporary Test Session',
    'description': 'This session will be deleted',
    'project': 'your-project-id'  # Replace with your own project ID
}

# Create the session
created_session = client.save_model('session', data=new_session_data).json()

if 'session' in created_session:
    session_id = created_session['session']['id']
    print(f"Created temporary session with ID: {session_id}")
    
    # Now delete the session we just created
    delete_response = client.delete_model("session", id=session_id)
    
    # Check if deletion was successful
    if delete_response.status_code == 204:  # 204 No Content indicates success
        print("Session deleted successfully")
    else:
        print(f"Delete failed with status code: {delete_response.status_code}")
"""

print("To use it, you would need to have write permissions and valid IDs.")
print("Only delete data that you have created and have permission to remove.")